In [72]:
from numpy.random import choice
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

In [73]:
data = pd.read_csv(r'C:\Users\okruz\Downloads\retail_train\retail_train.csv')
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [74]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity.head()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


In [75]:
popularity['weight'] = popularity['sales_value']

In [76]:
popularity=popularity.drop('sales_value',axis=1)

In [77]:
popularity.head()

,item_id,weight
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


In [78]:
popularity['weight'] = popularity['weight']/(sum(popularity['weight']))

In [79]:
result = data.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."


In [80]:
popularity.drop_duplicates

<bound method DataFrame.drop_duplicates of         item_id        weight
0         25671  2.817523e-06
1         26081  1.332067e-07
2         26093  2.139380e-07
3         26190  2.072103e-07
4         26355  2.664133e-07
...         ...           ...
89046  17991689  3.350349e-07
89047  17991691  3.350349e-07
89048  18000012  2.685661e-06
89049  18024155  5.368632e-07
89050  18024556  4.105187e-06

[89051 rows x 2 columns]>

In [81]:
p=popularity['weight']

In [82]:
p = np.array(p)

In [83]:
p /= sum(p)

In [84]:
def weighted_random(items, p, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, p=p, replace=False)
    
    return recs.tolist()

In [85]:
%%time

items = popularity['item_id']

result['weighted_random'] = result['user_id'].apply(lambda x: weight_random(items, p))

Wall time: 5.55 s


In [120]:
result.head(2)

,user_id,actual,weighted_random,compared
0,1,"[825123, 831447, 840361, 845307, 852014, 85498...","[861279, 5995177, 970866, 7168995, 13513120]",0.0
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55...","[907631, 5567098, 895295, 6534178, 1012873]",0.0


In [111]:
result['compared'] = 0.0

In [118]:
for i in range(len(result['weighted_random'])):
    resulted = [x for x in result['weighted_random'][i] if x in result['actual'][i]]
    result['compared'][i] = len(resulted)/5

<ipython-input-118-aaadb7fff1b9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['compared'][i] = len(resulted)/5


In [122]:
result['compared'].mean()

0.1005202080832321

In [123]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [124]:
result.apply(lambda row: precision_at_k(row['weighted_random'], row['actual']), axis=1).mean()

0.1005202080832321

2. Можно попробовать определить косинусную схожесть товаров пользователей, разбить их на группы по KNN и рекомендовать им самые популярные товары в их группе